# Artist classification by painting

This project explores the merits of using machine learning in 
supplementing art experts’ efforts in identifying forgeries among disputed paintings. From the movement of brushstrokes to the use of light and dark, successful algorithms will likely incorporate many aspects of a painter's unique style. 

[Multi Label Image Classification Simplified Model](https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/)

# Explanatory Data Analysis

### Download data

We will start by using the "train.zip" data [Painter by Numbers](https://www.kaggle.com/competitions/painter-by-numbers/overview) from Kaggle.As you can see the train.zip is 38.7 GB. That is a lot and will take a lot of time to download/load. 

For our timeline, instead of using the full dataset we will use a sub portion of the full train data: "train_1.zip". 

In [ ]:
# Import all libraries
import PIL as Image
import pandas as pd

Let's take a look at the first painting

In [ ]:
im = Image.open(r'C:\Users\14794\Documents\NACME_capstone\train_1\train_1\1.jpg')

Let's take a look of the first 10 images in the train_1 folder

In [ ]:
# List all files in the train_1 folder

list_dir = os.listdir('train_1/train_1/')

for i in len(list_dir):
  if i < 10:
    i.open(r'C:/Users/14794/Documents/NACME_capstone/train_1/train_1/' + i)
  else:
    break


Take notice how the images are ordered and sorted. We can see that the images don't go completely in order from 1.2.3.. and so on. Instead it goes 1.10.11.12.14... and so on. Next load in all_data_info.csv.

In [ ]:
# Load the data
labels = pd.read_csv('C:/Users/14794/Documents/NACME_capstone/all_data_info.csv/all_data_info.csv')
labels.head(10)

Viewing the first 10 samples, we can see that this dataframe contains all the image in all the train and test dataset. We will need to sort through the dataframe and use only rows corresponding to our train_1 dataset. 

In [ ]:
list_df = pd.DataFrame(list_dir, columns = ['new_filename'])
for i in list_df['new_filename']:
    list_df['path'] = path + i
print(list_df)

We labeled the list_dir column name "new_filename", since our code will need a column in common to merge efficiently. We also decided to append the path into the dataframe for future purposes if needed. 

In [ ]:
new_df = pd.merge(labels, list_df, on ="new_filename")
new_df = new_df.dropna()

By dropping all our naN values, our data lost roughly 2000 rows of data.

**Here we can add additional images to the train dataset, by using train_2 or other train files.**